## References 
- https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/


- https://machinelearningmastery.com/multivariate-time-series-forecasting-lstms-keras/



## Multivariate LSTM Models
Multivariate time series data means data where there is more than one observation for each time step.

There are two main models that we may require with multivariate time series data; they are:

- Multiple Input Series.
- Multiple Parallel Series.

## Import Library

In [1]:
import numpy as np

## Multiple Input Series
A problem may have two or more parallel input time series and an output time series that is dependent on the input time series.

The input time series are parallel because each series has an observation at the same time steps.

We can demonstrate this with a simple example of two parallel input time series where the output series is the simple addition of the input series

In [2]:
input_sequence_1=np.array([10,20,30,40,50,60,70,80,90])
input_sequence_2=np.array([15,25,35,45,55,65,75,85,95])
output_sequence=np.array([input_sequence_1[i]+input_sequence_2[i] for i in range(len(input_sequence_1))])
output_sequence

array([ 25,  45,  65,  85, 105, 125, 145, 165, 185])

In [3]:
print('shape: ',output_sequence.shape)
print('len: ',len(output_sequence))

shape:  (9,)
len:  9


We can reshape these three arrays of data as a single dataset where each row is a time step, and each column is a separate time series. This is a standard way of storing parallel time series in a CSV file.

In [4]:
input_sequence_1_reshaped=input_sequence_1.reshape((len(input_sequence_1),1))
input_sequence_2_reshaped=input_sequence_2.reshape((len(input_sequence_2),1))
output_sequence_reshaped=output_sequence.reshape((len(output_sequence),1))
print(output_sequence_reshaped.shape)
print(output_sequence_reshaped)

(9, 1)
[[ 25]
 [ 45]
 [ 65]
 [ 85]
 [105]
 [125]
 [145]
 [165]
 [185]]


In [5]:
# horizontall stack columns
from numpy import hstack
dataset=hstack((input_sequence_1_reshaped,input_sequence_2_reshaped,output_sequence_reshaped))
print(dataset.shape)
print(dataset)

(9, 3)
[[ 10  15  25]
 [ 20  25  45]
 [ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]
 [ 90  95 185]]


In [6]:
dataset[:,2]

array([ 25,  45,  65,  85, 105, 125, 145, 165, 185])

In [7]:
dataset[0:3,:-1]

array([[10, 15],
       [20, 25],
       [30, 35]])

In [8]:
dataset[3-1,-1]

65

We can define a function named split_sequences() that will take a dataset as we have defined it with rows for time steps and columns for parallel series and return input/output samples.

In [9]:
# split a multivariate sequence into samples
def split_sequence(sequence,n_steps):
    X,y=list(),list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix=i+n_steps
        # check if we are beyond the sequence
        if end_ix>len(sequence):
            break
        # gather input and output parts of the pattern
        seq_x,seq_y=sequence[i:end_ix,:-1],sequence[end_ix-1,-1]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X),np.array(y)

We can test this function on our dataset using three time steps for each input time series as input.

In [10]:
n_steps=3
X,y=split_sequence(dataset,n_steps)
for i in range(len(X)):
    print(X[i],y[i])

[[10 15]
 [20 25]
 [30 35]] 65
[[20 25]
 [30 35]
 [40 45]] 85
[[30 35]
 [40 45]
 [50 55]] 105
[[40 45]
 [50 55]
 [60 65]] 125
[[50 55]
 [60 65]
 [70 75]] 145
[[60 65]
 [70 75]
 [80 85]] 165
[[70 75]
 [80 85]
 [90 95]] 185


In [11]:
print(X.shape)
X

(7, 3, 2)


array([[[10, 15],
        [20, 25],
        [30, 35]],

       [[20, 25],
        [30, 35],
        [40, 45]],

       [[30, 35],
        [40, 45],
        [50, 55]],

       [[40, 45],
        [50, 55],
        [60, 65]],

       [[50, 55],
        [60, 65],
        [70, 75]],

       [[60, 65],
        [70, 75],
        [80, 85]],

       [[70, 75],
        [80, 85],
        [90, 95]]])

In [12]:
print(y.shape)
y

(7,)


array([ 65,  85, 105, 125, 145, 165, 185])

We can see that the X component has a three-dimensional structure.

The first dimension is the number of samples, in this case 7. The second dimension is the number of time steps per sample, in this case 3, the value specified to the function. Finally, the last dimension specifies the number of parallel time series or the number of variables, in this case 2 for the two parallel series.

This is the exact three-dimensional structure expected by an LSTM as input. The data is ready to use without further reshaping.

We will use a Vanilla LSTM where the number of time steps and parallel series (features) are specified for the input layer via the input_shape argumen

In [20]:
n_features=X.shape[2]
n_steps=3
input_shape=(n_steps,n_features)
input_shape

(3, 2)

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense


In [24]:
multiple_input_series_model=Sequential()
multiple_input_series_model.add(LSTM(50,activation='relu',input_shape=input_shape))
multiple_input_series_model.add(Dense(1))

multiple_input_series_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 50)                10600     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 51        
Total params: 10,651
Trainable params: 10,651
Non-trainable params: 0
_________________________________________________________________


In [25]:
multiple_input_series_model.compile(optimizer='adam',loss='mse')

In [28]:
history=multiple_input_series_model.fit(X,y,epochs=200,verbose=1)

Epoch 1/200
7/7 [==============================] - 0s 712us/sample - loss: 0.0974
Epoch 2/200
7/7 [==============================] - 0s 997us/sample - loss: 0.0993
Epoch 3/200
7/7 [==============================] - 0s 712us/sample - loss: 0.0959
Epoch 4/200
7/7 [==============================] - 0s 712us/sample - loss: 0.0973
Epoch 5/200
7/7 [==============================] - 0s 1ms/sample - loss: 0.0947
Epoch 6/200
7/7 [==============================] - 0s 712us/sample - loss: 0.0951
Epoch 7/200
7/7 [==============================] - 0s 998us/sample - loss: 0.0936
Epoch 8/200
7/7 [==============================] - 0s 997us/sample - loss: 0.0930
Epoch 9/200
7/7 [==============================] - 0s 855us/sample - loss: 0.0924
Epoch 10/200
7/7 [==============================] - 0s 712us/sample - loss: 0.0912
Epoch 11/200
7/7 [==============================] - 0s 570us/sample - loss: 0.0911
Epoch 12/200
7/7 [==============================] - 0s 712us/sample - loss: 0.0895
Epoch 13/200
7/

In [29]:
x_input=np.array([[70,75],[80,85],[90,95]])
x_input_reshaped=x_input.reshape((1,n_steps,n_features))
x_input_reshaped

array([[[70, 75],
        [80, 85],
        [90, 95]]])

In [30]:
y_predicted_multiple_input_series_model=multiple_input_series_model.predict(x_input_reshaped,verbose=0)
y_predicted_multiple_input_series_model

array([[185.17955]], dtype=float32)

In [31]:
print('Expected: %d' % (x_input[2,0]+x_input[2,1]))

Expected: 185


In [33]:
print('input:\n',x_input)
print('output:',y_predicted_multiple_input_series_model)

input:
 [[70 75]
 [80 85]
 [90 95]]
output: [[185.17955]]


## Multiple Parallel Series
A problem may have two or more parallel input time series and an output time series that is dependent on the input time series.

An alternate time series problem is the case where there are multiple parallel time series and a value must be predicted for each.

In [35]:
dataset

array([[ 10,  15,  25],
       [ 20,  25,  45],
       [ 30,  35,  65],
       [ 40,  45,  85],
       [ 50,  55, 105],
       [ 60,  65, 125],
       [ 70,  75, 145],
       [ 80,  85, 165],
       [ 90,  95, 185]])

We may want to predict the value for each of the three time series for the next time step.

This might be referred to as multivariate forecasting.

Again, the data must be split into input/output samples in order to train a model.

The first sample of this dataset would be:

In [53]:
print('sample input: \n',dataset[0:3])
print('sample output: ',dataset[3,:])

sample input: 
 [[10 15 25]
 [20 25 45]
 [30 35 65]]
sample output:  [40 45 85]


The split_sequences() function below will split multiple parallel time series with rows for time steps and one series per column into the required input/output shape

In [60]:
# split a multivariate sequence into samples
def split_sequence(sequence,n_steps):
    X,y=list(),list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix=i+n_steps
  
        # check if we are beyond the sequence
        if end_ix>len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x,seq_y=sequence[i:end_ix],sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X),np.array(y)

In [62]:
n_steps=3
X,y=split_sequence(dataset,n_steps)
for i in range(len(X)):
    print(X[i],y[i])

[[10 15 25]
 [20 25 45]
 [30 35 65]] [40 45 85]
[[20 25 45]
 [30 35 65]
 [40 45 85]] [ 50  55 105]
[[ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]] [ 60  65 125]
[[ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]] [ 70  75 145]
[[ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]] [ 80  85 165]
[[ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]] [ 90  95 185]


In [65]:
print(X.shape)
X

(6, 3, 3)


array([[[ 10,  15,  25],
        [ 20,  25,  45],
        [ 30,  35,  65]],

       [[ 20,  25,  45],
        [ 30,  35,  65],
        [ 40,  45,  85]],

       [[ 30,  35,  65],
        [ 40,  45,  85],
        [ 50,  55, 105]],

       [[ 40,  45,  85],
        [ 50,  55, 105],
        [ 60,  65, 125]],

       [[ 50,  55, 105],
        [ 60,  65, 125],
        [ 70,  75, 145]],

       [[ 60,  65, 125],
        [ 70,  75, 145],
        [ 80,  85, 165]]])

In [66]:
print(y.shape)
y

(6, 3)


array([[ 40,  45,  85],
       [ 50,  55, 105],
       [ 60,  65, 125],
       [ 70,  75, 145],
       [ 80,  85, 165],
       [ 90,  95, 185]])

In [69]:
n_steps=3
n_features=X.shape[1]
input_shape=(n_steps,n_features)
input_shape

(3, 3)

We will use a Stacked LSTM where the number of time steps and parallel series (features) are specified for the input layer via the input_shape argument. The number of parallel series is also used in the specification of the number of values to predict by the model in the output layer

In [77]:
multiple_input_parallel_series_model=Sequential()
multiple_input_parallel_series_model.add(LSTM(100,activation='relu',return_sequences=True,input_shape=input_shape))
multiple_input_parallel_series_model.add(LSTM(100,activation='relu'))
multiple_input_parallel_series_model.add(Dense(n_features))

multiple_input_parallel_series_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 3, 100)            41600     
_________________________________________________________________
lstm_9 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_6 (Dense)              (None, 3)                 303       
Total params: 122,303
Trainable params: 122,303
Non-trainable params: 0
_________________________________________________________________


In [78]:
multiple_input_parallel_series_model.compile(optimizer='adam',loss='mse')

In [79]:
history=multiple_input_parallel_series_model.fit(X,y,epochs=200,verbose=1)

Epoch 1/200
6/6 [==============================] - 3s 448ms/sample - loss: 9608.4463
Epoch 2/200
6/6 [==============================] - 0s 997us/sample - loss: 9396.3164
Epoch 3/200
6/6 [==============================] - 0s 1ms/sample - loss: 9213.1562
Epoch 4/200
6/6 [==============================] - 0s 1ms/sample - loss: 9033.3057
Epoch 5/200
6/6 [==============================] - 0s 2ms/sample - loss: 8850.6729
Epoch 6/200
6/6 [==============================] - 0s 1ms/sample - loss: 8664.3145
Epoch 7/200
6/6 [==============================] - 0s 1ms/sample - loss: 8452.8496
Epoch 8/200
6/6 [==============================] - 0s 2ms/sample - loss: 8236.7979
Epoch 9/200
6/6 [==============================] - 0s 1ms/sample - loss: 8004.4341
Epoch 10/200
6/6 [==============================] - 0s 1ms/sample - loss: 7698.5210
Epoch 11/200
6/6 [==============================] - 0s 1ms/sample - loss: 7342.9858
Epoch 12/200
6/6 [==============================] - 0s 1ms/sample - loss: 6827.14

## Evaluation the model

In [82]:
x_input=np.array([[70,75,145],[80,85, 165],[90,95, 185]])
x_input_reshaped=x_input.reshape((1,n_steps,n_features))
print(x_input_reshaped.shape)
x_input_reshaped

(1, 3, 3)


array([[[ 70,  75, 145],
        [ 80,  85, 165],
        [ 90,  95, 185]]])

In [83]:
y_predicted_multiple_input_parallel_series = multiple_input_parallel_series_model.predict(x_input_reshaped)
y_predicted_multiple_input_parallel_series

array([[102.03651, 107.80541, 211.25662]], dtype=float32)

In [95]:
print('Expected: %d %d %d' % (x_input[2,0]+10,x_input[2,1]+10,x_input[2,0]+x_input[2,1]+20))

Expected: 100 105 205
